# VacationPy
-----

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store WeatherPy Results in DataFrame
* Load the cities.csv exported in WeatherPy to a DataFrame

In [6]:
# Store csv created in WeatherPy into a DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ponta do sol,32.6667,-17.1000,57.70,70,39,2.44,PT,1647836727
1,1,port elizabeth,-33.9180,25.5701,66.31,98,75,2.30,ZA,1647836727
2,2,lebu,-37.6167,-73.6500,56.08,94,25,5.64,CL,1647836728
3,3,barranca,-10.7500,-77.7667,69.75,82,98,6.42,PE,1647836728
4,4,castro,-24.7911,-50.0119,56.88,91,91,9.60,BR,1647836728


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [8]:
# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions
* Drop any rows will null values

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
75,75,multan,30.1956,71.4753,75.09,73,0,0.00,PK,1647836643
96,96,salalah,17.0151,54.0924,73.49,16,0,1.14,OM,1647836768
254,254,manama,26.2154,50.5832,73.38,28,0,5.75,BH,1647836553
473,473,puerto escondido,15.8500,-97.0667,79.50,68,0,6.20,MX,1647836896


### Hotel Map
* Store into variable named `hotel_df`
* Add a "Hotel Name" column to the DataFrame
* Set parameters to search for hotels within 5000 meters
* Hit the Google Places API for each city's coordinates
* Store the first Hotel result into the DataFrame
* Plot markers on top of the heatmap

In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
75,multan,PK,30.1956,71.4753,
96,salalah,OM,17.0151,54.0924,
254,manama,BH,26.2154,50.5832,
473,puerto escondido,MX,15.8500,-97.0667,


In [11]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
75,multan,PK,30.1956,71.4753,
96,salalah,OM,17.0151,54.0924,
254,manama,BH,26.2154,50.5832,
473,puerto escondido,MX,15.8500,-97.0667,


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))